In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Dropout, MaxPool2D, Conv2D, Flatten, Rescaling, Resizing
from tensorflow.keras.models import Sequential, Model

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from collections import Counter

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    tf.config.experimental.enable_growth()
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

AttributeError: module 'tensorflow._api.v2.config.experimental' has no attribute 'enable_growth'

In [ ]:
! python -c "import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))"

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
! nvidia-smi

In [ ]:
# ! pip install kaggle
# ! mkdir ~/.kaggle
# ! cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle datasets download paultimothymooney/chest-xray-pneumonia
# ! unzip /content/chest-xray-pneumonia.zip

# Project Notebook :)

## Main ideas

* time: 10.jan.2024 - 5.feb.2024
* has to be simple
* maybe somethings about biology

### Objective

Create a robust model that detects pediatric pnuemonia. By robust, I mean a model that isn't effected by rotations (like in [situs inversus](https://en.wikipedia.org/wiki/Situs_inversus), mirrored organs; or when a doctor may flip the x-ray picture), different brightness levels and other.

I have downloaded a [chest x-ray ds](https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia/data) that can be used to identify pneumonia patients.

First I will create a simple CNN that can detedect the raw data from the ds. Then I would like to build on top of that by merging already trained NN (like ResNet101 or another, more specialized for medical/x-ray image classification) to create a model that generalizes better.

**Objectives:**
* [x] Wacth 2 lectures on NN testing and how to work with images

**Main objectives**
1. Load the data and create a simple CNN
  1. [x] Load images
  2. [x] Apply normalization
  3. [x] Create a CNN and train it.
  4. [ ] Discuss it
2.






### Load data

First I want to load my data from the mentioned ds. I will be using `tf.keras.utils.image_dataset_from_directory` to do so. The sets are split into 80/20 for training and vaidation. Another set has been separated beforehand for testing which has 624 images.

In [16]:
DATA_PATH = "../data/"
BATCH_SIZE = 16
AUTOTUNE = tf.data.AUTOTUNE
CLASS_NUM = 11
IMG_SIZE = 128

In [17]:
data = tf.keras.utils.image_dataset_from_directory(DATA_PATH,
                                                    batch_size = BATCH_SIZE,
                                                    image_size = (IMG_SIZE, IMG_SIZE),
                                                    shuffle = True,
                                                    seed = 42)

Found 12210 files belonging to 11 classes.


In [18]:
TRAIN_SAMPLES = 100
TEST_VAL_SAMPLES = np.round(TRAIN_SAMPLES * 0.2)

train_set = data.take(TRAIN_SAMPLES)

validation_set = data.skip(TRAIN_SAMPLES).take(TEST_VAL_SAMPLES)

test_set = data.skip(TRAIN_SAMPLES).skip(TEST_VAL_SAMPLES)

The model is constructed by 4 convolutional layers with valid padding. After every conv layer there is a pool layer to optimize training time.

Flatten layer is used to feed the data into the dense layers at the end.

We also rescale the pixels (`Rescaling(1./255)`) into range [0, 1] as a form of normalization.

Hidden layers are using `relu` for activation since the function's efficiency and at the output I have set 'sigmod' for classification of 2 classes. Sigmoid is used since it gives a range of [0, 1] which can be used as a *probability* metric.

In [ ]:
class_counts = Counter()
for images, labels in data:
    # Extract the labels from the batch of labels
    labels = labels.numpy()
  
    # Increment the count of each class label
    for label in labels:
        class_counts[label] += 1

In [ ]:
class_counts

In [ ]:
class_names = data.class_names
x = []
for el in class_counts.keys():
    x.append(class_names[el])
y = class_counts.values()

In [ ]:
plt.figure(figsize= (20,5))
plt.bar(x, y)


plt.figure(figsize=(10, 10))
# Access every img and label from the ds
for images, labels in data.take(1):

    #   
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")
        print(labels[i])
        break

    break

In [19]:
tf.keras.backend.clear_session()

In [20]:
model = Sequential([
    Input(shape = (IMG_SIZE, IMG_SIZE, 3)),

    Rescaling(1./255),

    Conv2D(32, 3, padding = 'valid', activation = 'relu'),
    MaxPool2D(),
    Conv2D(16, 3, padding = 'valid', activation = 'relu'),
    MaxPool2D(),

    Flatten(),
    Dense(32, activation = 'relu'),
    Dropout(0.2),
    Dense(32, activation = 'relu'),
    Dense(16, activation = 'relu'),
    # OUTPUT
    Dense(CLASS_NUM, activation = 'softmax'),
])

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

model.fit(train_set, validation_data = validation_set, epochs = 100)

Epoch 1/100
100/100 [==============================] - 8s 72ms/step - loss: 2.1995 - accuracy: 0.2313 - val_loss: 2.0731 - val_accuracy: 0.2875
Epoch 2/100
100/100 [==============================] - 7s 70ms/step - loss: 2.0910 - accuracy: 0.2525 - val_loss: 2.0972 - val_accuracy: 0.2719
Epoch 3/100
100/100 [==============================] - 7s 70ms/step - loss: 2.0837 - accuracy: 0.2781 - val_loss: 2.0414 - val_accuracy: 0.2969
Epoch 4/100
100/100 [==============================] - 8s 75ms/step - loss: 2.0531 - accuracy: 0.2856 - val_loss: 2.0834 - val_accuracy: 0.2812
Epoch 5/100
100/100 [==============================] - 9s 85ms/step - loss: 2.0271 - accuracy: 0.3137 - val_loss: 2.0517 - val_accuracy: 0.3063
Epoch 6/100
100/100 [==============================] - 7s 71ms/step - loss: 2.0473 - accuracy: 0.2962 - val_loss: 2.1636 - val_accuracy: 0.2562
Epoch 7/100
100/100 [==============================] - 7s 70ms/step - loss: 2.0188 - accuracy: 0.3075 - val_loss: 2.0704 - val_accuracy:

In [21]:
model.evaluate(train_set)

100/100 [==============================] - 4s 35ms/step - loss: 0.2412 - accuracy: 0.9244


[0.24117989838123322, 0.9243749976158142]

In [ ]:
model.save('pediatric_pneumonia_model_2.keras')

In [ ]:
model.summary()

#### Discussion

The simple model (229 938 params) has test and val accuracy of 0.96, which is in the range of BOE but that doesn't mean it generalizes better. IRL images can have different brightness and slight rotations which throw off the NN.

I would like now to train it with images with different brightness, rotations and horizontal positions. There are 2 options:

Option 1 is to add random brightness, flip and rotation layers in the model's architecture.

Option 2 is to upsample the data by simply having duplicates of images but with augmentations. This, I think, won't suffice since the model will be training on duplicates instead of single-cases.

For this reason I will add augmentation layers to the model.

### CNN with image augmentations

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
model = Sequential([
    Resizing(IMG_SIZE)
    Rescaling(1./255),

    Conv2D(32, 3, padding = 'valid', activation = 'relu'),
    MaxPool2D(),
    Conv2D(32, 3, padding = 'valid', activation = 'relu'),
    MaxPool2D(),

    Conv2D(32, 3, padding = 'valid', activation = 'relu'),
    MaxPool2D(),
    Conv2D(32, 3, padding = 'valid', activation = 'relu'),
    MaxPool2D(),

    Flatten(),
    Dense(32, activation = 'relu'),
    Dense(16, activation = 'relu'),
    Dense(CLASS_NUM, activation = 'sigmoid'),
])